# Predictions of arutema47

In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
import os
from PIL import Image
import pandas as pd
from tqdm.auto import tqdm

debug = True # if true, run all public data for action.

In [ ]:
df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
if (df.shape[0] == 2477) and (debug==False):
    fast_sub = True
    fast_df = pd.DataFrame(([['00086460a852_study', 'negative 1 0 0 1 1'], 
                         ['000c9c05fd14_study', 'negative 1 0 0 1 1'], 
                         ['65761e66de9f_image', 'none 1 0 0 1 1'], 
                         ['51759b5579bc_image', 'none 1 0 0 1 1']]), 
                       columns=['id', 'PredictionString'])
else:
    fast_sub = False

In [ ]:
files = []
split = "test"

if not fast_sub:
    for studyname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        for file in filenames:
            files.append({"img_id":file.split(".")[0], "study_id":studyname.split("/")[-2]})
            #print(file, studyname.split("/")[-2])
else:
    files.append({"img_id": "65761e66de9f", "study_id": "00086460a852"})
    files.append({"img_id": "51759b5579bc", "study_id": "000c9c05fd14"})
df_test = pd.DataFrame(files)
df_test

In [ ]:
df_test

# .dcm to .png

In [ ]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
# make image imgs only to save time.
split = 'test'
save_dir = f'/kaggle/tmp/{split}/'

os.makedirs(save_dir, exist_ok=True)

save_dir = f'/kaggle/tmp/{split}/study/'
os.makedirs(save_dir, exist_ok=True)
image_id = []
dim0 = []
dim1 = []
splits = []
save_dir = f'/kaggle/tmp/{split}/image/'
os.makedirs(save_dir, exist_ok=True)

if fast_sub:
    xray = read_xray('../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
    im = resize(xray, size=512)  
    im.save(os.path.join(save_dir,'65761e66de9f_image.png'))
    image_id.append('65761e66de9f.dcm'.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)
    xray = read_xray('../input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
    im = resize(xray, size=512)  
    im.save(os.path.join(save_dir, '51759b5579bc_image.png'))
    image_id.append('51759b5579bc.dcm'.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)
else:
    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=512)  
            im.save(os.path.join(save_dir, file.replace('.dcm', '_image.png')))
            image_id.append(file.replace('.dcm', ''))
            dim0.append(xray.shape[0])
            dim1.append(xray.shape[1])
            splits.append(split)
            
meta = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits})

In [ ]:
# meta image data for detections
meta

In [ ]:
!ls /kaggle/tmp/test/image

# study predict

In [ ]:
import numpy as np 
import pandas as pd
df = df_test

study_len = len(df.study_id.unique())

imsize = 384
study_len

In [ ]:
df

In [ ]:
study_df = df
study_df

In [ ]:
!pip install ../input/d/kyoshioka47/siim-arutema/pytorch-image-models -q

In [ ]:
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset

def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )


def up_conv(in_channels, out_channels):
    return nn.ConvTranspose2d(
        in_channels, out_channels, kernel_size=2, stride=2
    )


class unet(nn.Module):
    def __init__(self, kernel_type="swin", out_channels=1, pretrained = False):
        super(unet, self).__init__()
        self.kernel_type = kernel_type
        if "swinl" in kernel_type:
            self.back = timm.create_model("swin_large_patch4_window12_384",pretrained=pretrained, return_dense=True, num_classes=0,)
            ch = 1536
            self.start = nn.Sequential(*list(self.back.children())[:-3])
        elif "swin" in kernel_type:
            self.back = timm.create_model("swin_base_patch4_window12_384",pretrained=pretrained, return_dense=True, num_classes=0,)
            ch = 1024
            self.start = nn.Sequential(*list(self.back.children())[:-3])
        elif "effv2l" in kernel_type:
            self.back = timm.create_model("tf_efficientnetv2_l_in21ft1k", pretrained=pretrained,  num_classes=0,)
            ch = 1280
            self.start = nn.Sequential(*list(self.back.children())[:-2])      
        elif "effv2s" in kernel_type:
            self.back = timm.create_model("tf_efficientnetv2_s_in21ft1k", pretrained=pretrained,  num_classes=0,)
            ch = 1280
            self.start = nn.Sequential(*list(self.back.children())[:-2])        
        
        elif "eff" in kernel_type:
            self.back = timm.create_model("tf_efficientnetv2_m_in21ft1k", pretrained=pretrained,  num_classes=0,)
            ch = 1280
            self.start = nn.Sequential(*list(self.back.children())[:-2])
        elif "res50" in kernel_type:
            self.back = timm.create_model("resnet50d", pretrained=pretrained,  num_classes=0,)
            ch = 2048
            self.start = nn.Sequential(*list(self.back.children())[:-2])
        
        self.ch = ch
        self.up_conv1 = up_conv(ch, 512)
        self.double_conv1 = double_conv(512, 256)
        
        self.do = nn.Dropout(0.5)
        self.fc = nn.Linear(ch, 4)
        
        self.do2 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(ch, 3)

        self.final_conv = nn.Conv2d(256, out_channels, kernel_size=1)
        
    def forward(self, x):
        input_ = x
        x = self.start(x)
        # classifier head        
        if "swin" in self.kernel_type:
            tmp = x.reshape(-1, self.ch, 12, 12)            
        else:
            tmp = x
        
        # cls head
        cls = F.adaptive_avg_pool2d(tmp, 1).squeeze(-1).squeeze(-1)
        cls = self.do(cls)
        cls = self.fc(cls)
        
        # cls head2
        cls2 = F.adaptive_avg_pool2d(tmp, 1).squeeze(-1).squeeze(-1)
        cls2 = self.do2(cls2)
        cls2 = self.fc2(cls2)

        # seg head
        x = self.up_conv1(tmp)
        x = self.double_conv1(x)
        x = self.final_conv(x)
            
        return x, cls, cls2

In [ ]:
model = unet()
model(torch.rand(1,3,384,384))[0].size()

In [ ]:
class cassavaDataset(Dataset):
    def __init__(self,
                 df,
                 rand=False,
                 transform=None,
                ):

        self.df = df.reset_index(drop=True)
        self.rand = rand
        self.transform = transform

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_id = row.img_id
        
        tiff_file = os.path.join(image_folder, f'{img_id}_image.png')
        images = cv2.imread(tiff_file)   
        
        # aug
        if self.transform is not None:
            out = self.transform(image=images,)
            images = out["image"]
              
        images = images.astype(np.float32)
        images /= 255
        images = images.transpose(2, 0, 1)
        
        return torch.tensor(images), row.study_id, row.img_id
    
import albumentations
import albumentations as A

transforms_val = albumentations.Compose([albumentations.Resize(imsize, imsize, p=1.0)])

In [ ]:
from sklearn.metrics import average_precision_score
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
import torch.nn.functional as F
import cv2

def val_epoch(loader, get_output=True):
    model.eval()
    val_loss = []
    LOGITS = []
    PREDS = []
    TARGETS = []
    study = []
    img = []

    with torch.no_grad():
        for (data, s, i) in tqdm(loader):
            data = data.to(device)
            _, logits, _ = model(data)
            _, logits2, _ = model(data.flip(3))
            logits = logits.squeeze(1)
            logits2 = logits2.squeeze(1)
            logits = (logits+logits2)/2
            study.extend(s)
            img.extend(i)
            
            pred = F.softmax(logits).detach()
            LOGITS.append(logits)
            PREDS.append(pred)

    LOGITS = torch.cat(LOGITS).cpu().numpy()   
    PREDS = torch.cat(PREDS).cpu().numpy()
    PREDS_ARG = PREDS.argmax(1)
    
    if get_output:
        return LOGITS, PREDS, study, img
    else:
        return val_loss, acc, np.mean(map)

In [ ]:
########### infer swin
imsize = 384
image_folder = "/kaggle/tmp/test/image/"
device ="cuda"

transforms_val = albumentations.Compose([albumentations.Resize(imsize, imsize, p=1.0)])
dataset_valid = cassavaDataset(study_df, transform=transforms_val)
# Setup dataloader
valid_loader = torch.utils.data.DataLoader(dataset_valid, batch_size=16, sampler=SequentialSampler(dataset_valid), num_workers=2)

# Initialize model
preds = []

for fold in range(4):
    model = unet("swin")
    model = model.to(device)
    model.load_state_dict(torch.load("../input/siimfinal/unet-swin-4class-clean_fold{}_imsize384_dec0_scaler0.5_smooth_best_r7_fold{}.pth".format(fold, fold)))

    LOGITS, PREDS, study, image  = val_epoch(valid_loader)
    preds.append(PREDS)
    
for fold in range(5):
    kernel_type = "swin"
    model = unet("swin")
    model = model.to(device)
    model.load_state_dict(torch.load("../input/siimfinal/unet-swin-4class-r8_fold{}_imsize384_dec0_scaler0.5_smooth_best_r7_fold{}.pth".format(fold, fold)))

    LOGITS, PREDS, study, image  = val_epoch(valid_loader)
    preds.append(PREDS)

In [ ]:
kernel_type = "eff"
transforms_val = albumentations.Compose([albumentations.Resize(512, 512, p=1.0)])
dataset_valid = cassavaDataset(study_df, transform=transforms_val)
# Setup dataloader
valid_loader = torch.utils.data.DataLoader(dataset_valid, batch_size=8, sampler=SequentialSampler(dataset_valid), num_workers=2)

# memory clean
import gc
gc.collect()

for fold in range(4):
    model = unet("eff")
    model = model.to(device)
    model.load_state_dict(torch.load("../input/siimfinal/unet-eff-4class-clean_fold{}_imsize512_dec0_scaler0.5_smooth_best_r7_fold{}.pth".format(fold, fold)))

    LOGITS, PREDS, study, image  = val_epoch(valid_loader)
    preds.append(PREDS)

In [ ]:
# memory clean
import gc
gc.collect()

kernel_type = "eff"
transforms_val = albumentations.Compose([albumentations.Resize(512, 512, p=1.0)])
dataset_valid = cassavaDataset(study_df, transform=transforms_val)
# Setup dataloader
valid_loader = torch.utils.data.DataLoader(dataset_valid, batch_size=8, sampler=SequentialSampler(dataset_valid), num_workers=2)
    
for fold in range(5):
    model = unet("eff")
    model = model.to(device)
    model.load_state_dict(torch.load("../input/siimfinal/unet-effv2m-4class-r8_fold{}_imsize512_dec0_scaler0.5_smooth_best_r7_fold{}.pth".format(fold, fold)))

    LOGITS, PREDS, study, image  = val_epoch(valid_loader)
    preds.append(PREDS)

In [ ]:
del model, valid_loader

# memory clean
import gc
gc.collect()

## Process study inference

In [ ]:
PREDS = np.mean(np.array(preds),axis=0)
PREDS[:5]

In [ ]:
def get_PredictionString(row, thr=0):
    string = ''
    for idx in range(4):
        conf =  row[str(idx)]
        if conf>thr:
            string+=f'{label2name[idx]} {conf:0.2f} 0 0 1 1 '
    string = string.strip()
    return string

name2label = { 
    'negative': 0,
    'indeterminate': 2,
    'atypical': 3,
    'typical': 1}
label2name  = {v:k for k, v in name2label.items()}

In [ ]:
outs = []
for i in range(len(study)):
    out = {"id": study[i]+"_study", "img_id": image[i]+"_image", "0": PREDS[i][3], "1": PREDS[i][0], "2": PREDS[i][1], "3": PREDS[i][2]}
    outs.append(out)
outs[0]

In [ ]:
df = pd.DataFrame(outs) # store predictions as df
df_study = df # save for image predictions.

df = df[~df.id.duplicated()].reset_index(drop=True) # simply drop multi-images..(やりようはある気がするが時間ねえ)
df

In [ ]:
# make submission string
df['PredictionString'] = df.apply(get_PredictionString, axis=1)
df = df.drop(["0","1","2","3"], axis=1)
study_sub = df
df

In [ ]:
del preds, outs
# memory clean
import gc
gc.collect()

# Image Inference (yolov5)

In [ ]:
!pip install ../input/odachkaggle/ODA-Object-Detection-ttA-main -q

In [ ]:
################## boxes
from odach.wbf import *
import os
import cv2

def run_wbf(predictions, iou_thr=0.4, skip_box_thr=0.2, weights=None):
    boxes = [(prediction['boxes']).tolist()  for prediction in predictions]
    scores = [prediction['scores'].tolist()  for prediction in predictions]    
    labels = [prediction['labels'].tolist() for prediction in predictions]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    return boxes, scores, labels
          
def process_box(boxes, iou, skip):
    preds1 = {"boxes": boxes[:,2:],
                  "labels": boxes[:,0],
                  "scores": boxes[:,1]}
    preds = [preds1,]

    boxes, scores, labels = run_wbf(preds, iou_thr=iou, skip_box_thr=skip)
    return np.stack([labels.reshape(-1), scores.reshape(-1), boxes[:,0],boxes[:,1],boxes[:,2],boxes[:,3]]).T
          
def yolo2voc(bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes
          
def get_boxes(filepath,train=False,iou=0.5,skip=0.):
    if os.path.exists(filepath):
        w = 1
        h = 1
        # read results
        f = open(filepath, 'r')
        data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
        data = data[:, [0, 5, 1, 2, 3, 4]]

        # to voc
        bboxes = np.concatenate((data[:, :2], yolo2voc(data[:, 2:])), axis =1) 
        # nms
        bboxes = process_box(bboxes,iou,skip) 

        # resize boxes to original size
        bboxes[:,2] = w*bboxes[:,2]
        bboxes[:,4] = w*bboxes[:,4]
        bboxes[:,3] = h*bboxes[:,3]
        bboxes[:,5] = h*bboxes[:,5]
        return bboxes

In [ ]:
# get negative prediction from study models
import glob
files = sorted(glob.glob("/kaggle/tmp/test/image/*"))

# inference by study model
inferences = [f.split("/")[-1].split(".")[0] for f in files]
PREDS = [df_study[df_study.img_id==inferences[i]].reset_index()["0"][0] for i,_ in enumerate(inferences)]
PREDS[:5]

In [ ]:
meta

In [ ]:
# get meta info
meta = meta[meta['split'] == 'test']

if fast_sub:
    test_df = fast_df.copy()
else:
    test_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')

test_df = test_df[study_len:].reset_index(drop=True) 
print(test_df)
meta.columns = ['id', 'dim0', 'dim1', 'split']
test_df = pd.merge(test_df, meta, on = 'id', how = 'left')
test_df

In [ ]:
# Run yolov5 inference
dim = 512 #1024, 256, 'original'
test_dir = f'/kaggle/tmp/{split}/image'
import shutil
try:
    shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
except:
    print("oops")
    
os.chdir('/kaggle/working/yolov5') # install dependencies

import torch

for fold in range(5):
    for model in ["l6", "x6"]:
        weights_dir = '/kaggle/input/siimfinal/{}_{}.pt'.format(model, fold)
        name = "{}_{}".format(model, fold)
        !python detect.py --weights $weights_dir --name $name\
        --img 512\
        --conf 0.001\
        --iou 0.6\
        --source $test_dir\
        --save-txt --save-conf --exist-ok

In [ ]:
!ls runs/detect/l6_0/labels

In [ ]:
meta

### Boxes to predictions

In [ ]:
import glob

preds = []

for i,file in tqdm(enumerate(files)):
    filepath = file
    id = filepath.split("/")[-1].split(".")[0]
    print(id)
    bbox = []
    for fold in range(5):
        for model in ["l6","x6"]:
            path = 'runs/detect/{}_{}/labels/{}.txt'.format(model, fold, id)
            if os.path.exists(path):
                #try:
                    boxes = get_boxes(path,iou=1)
                    bbox.append({"boxes": boxes[:,2:],
                      "labels": boxes[:,0],
                      "scores": boxes[:,1]})
                #except:
                #    print("not found {}".format(path))
    # ensemble boxes
    boxes, bscores, labels = run_wbf(bbox, iou_thr=0.6, skip_box_thr=0.)
    box = np.stack([labels.reshape(-1), bscores.reshape(-1), boxes[:,0],boxes[:,1],boxes[:,2],boxes[:,3]]).T

    w = int(meta[meta.id+"_image"==id].dim0)
    h = int(meta[meta.id+"_image"==id].dim1)

    box[:,2] *= h
    box[:,4] *= h
    box[:,3] *= w
    box[:,5] *= w
    string = ""
    new = []
    for b in box:
        string += "opacity {} {} {} {} {} ".format(b[1], b[2], b[3], b[4], b[5])
    
    # append none
    string += "none {} {} {} {} {} ".format(PREDS[i], 0,0,1,1)
    preds.append({"id": id, "PredictionString": string})

In [ ]:
preds[0]

In [ ]:
df2 = pd.DataFrame(preds) # get image detections

# Prepare final submission

In [ ]:
sub = pd.concat([study_sub, df2]).reset_index(drop=True).drop(["img_id"], axis=1)
sub

In [ ]:
sub.to_csv('/kaggle/working/submission.csv',index = False)  
shutil.rmtree('/kaggle/working/yolov5')